In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.models.llama.modeling_llama import apply_rotary_pos_emb

/root/workspace/gnovack/vllm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [3]:
model.model.layers = model.model.layers[:1]
model = model.to(torch.bfloat16)
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm

In [7]:
input_ids = torch.tensor([    1, 15043, 29892,   590,  1024,   338,     1,   450,  6673,   310,
          278,  3303,  3900,   338,     1,   450,  7483,   310,  3444,   338,
            1,   450,  5434,   310,   319, 29902,   338,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],
       dtype=torch.int32).unsqueeze(0)


In [8]:
outputs = model(input_ids, output_hidden_states=True, output_attentions=True)

In [12]:
outputs.hidden_states[-1][0, :6, :]

tensor([[-0.1494, -0.8125,  1.8359,  ..., -0.5195, -1.1484, -1.3516],
        [-1.3359,  0.8125, -0.5938,  ...,  1.5391,  1.7188,  0.9023],
        [-0.9570,  0.4316, -0.4121,  ...,  0.0747,  0.4453, -0.0378],
        [ 0.9922, -1.5703,  1.7422,  ...,  0.3613,  0.2334,  1.2266],
        [-0.0067,  1.4609,  0.8281,  ..., -1.0234,  0.9375,  0.7969],
        [-1.1484,  1.3516, -0.0215,  ..., -0.5664, -0.6055,  3.0312]],
       dtype=torch.bfloat16, grad_fn=<SliceBackward0>)

In [8]:
attn_scores = logits.attentions[0]

In [9]:
embeds = model.model.embed_tokens(input_ids)
embeds

tensor([[[ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334],
         [ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334],
         [ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334],
         ...,
         [ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334],
         [ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334],
         [ 0.0031,  0.0178,  0.0210,  ..., -0.0052, -0.0420, -0.0334]]],
       dtype=torch.bfloat16, grad_fn=<EmbeddingBackward0>)

In [10]:
# input_shape = embeds.shape[:-1]
# hidden_shape = (*input_shape, -1, 64)
# k = model.model.layers[0].self_attn.k_proj(embeds)#.view(hidden_shape).transpose(1, 2)

norm_embeds = model.model.layers[0].input_layernorm(embeds)
norm_embeds


tensor([[[ 0.0271,  0.1826,  0.3027,  ..., -0.0635, -0.4902, -0.2812],
         [ 0.0271,  0.1826,  0.3027,  ..., -0.0635, -0.4902, -0.2812],
         [ 0.0271,  0.1826,  0.3027,  ..., -0.0635, -0.4902, -0.2812],
         ...,
         [ 0.0271,  0.1826,  0.3027,  ..., -0.0635, -0.4902, -0.2812],
         [ 0.0271,  0.1826,  0.3027,  ..., -0.0635, -0.4902, -0.2812],
         [ 0.0271,  0.1826,  0.3027,  ..., -0.0635, -0.4902, -0.2812]]],
       dtype=torch.bfloat16, grad_fn=<MulBackward0>)

In [42]:
input_shape = embeds.shape[:-1]
hidden_shape = (*input_shape, -1, 64)

q = model.model.layers[0].self_attn.q_proj(norm_embeds)
k = model.model.layers[0].self_attn.k_proj(norm_embeds)
v = model.model.layers[0].self_attn.v_proj(norm_embeds)

position_embeds = model.model.rotary_emb(embeds, torch.arange(0,128).unsqueeze(0))
attn_out = model.model.layers[0].self_attn(norm_embeds, position_embeddings=position_embeds)
print(attn_out[0])
attn_out = attn_out[0] + embeds
# print(attn_out)
attn_out_norm = model.model.layers[0].post_attention_layernorm(attn_out)
print(attn_out_norm)
mlp_out = model.model.layers[0].mlp(attn_out_norm)
print(mlp_out)

tensor([[[-0.0148,  0.0123,  0.0157,  ...,  0.0266,  0.0075, -0.0199],
         [-0.0148,  0.0123,  0.0157,  ...,  0.0266,  0.0075, -0.0199],
         [-0.0148,  0.0123,  0.0157,  ...,  0.0266,  0.0075, -0.0199],
         ...,
         [-0.0148,  0.0123,  0.0157,  ...,  0.0266,  0.0075, -0.0199],
         [-0.0148,  0.0123,  0.0157,  ...,  0.0266,  0.0075, -0.0199],
         [-0.0148,  0.0123,  0.0157,  ...,  0.0266,  0.0075, -0.0199]]],
       dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)
tensor([[[-0.1001,  0.2559,  0.2871,  ...,  0.1953, -0.2969, -0.4707],
         [-0.1001,  0.2559,  0.2871,  ...,  0.1953, -0.2969, -0.4707],
         [-0.1001,  0.2559,  0.2871,  ...,  0.1953, -0.2969, -0.4707],
         ...,
         [-0.1001,  0.2559,  0.2871,  ...,  0.1953, -0.2969, -0.4707],
         [-0.1001,  0.2559,  0.2871,  ...,  0.1953, -0.2969, -0.4707],
         [-0.1001,  0.2559,  0.2871,  ...,  0.1953, -0.2969, -0.4707]]],
       dtype=torch.bfloat16, grad_fn=<MulBackward0>)
ten

In [114]:
torch.matmul(attn_scores, v)

tensor([[[[-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          ...,
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275]],

         [[-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          ...,
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275]],

         [[-0.0038,  0.0505,  0.0369,  ...,  0.0034, -0.0287,  0.0275],
          [-0.0038,  0.0505,  

In [98]:
torch.einsum(
    'bsh,hq->bsq',
    norm_embeds,
    model.model.layers[0].self_attn.q_proj.weight.t()
)

tensor([[[ 0.8398,  2.2812,  2.7969,  ...,  5.7500,  0.9062, -3.4844],
         [ 0.8398,  2.2812,  2.7969,  ...,  5.7500,  0.9062, -3.4844],
         [ 0.8398,  2.2812,  2.7969,  ...,  5.7500,  0.9062, -3.4844],
         ...,
         [ 0.8398,  2.2812,  2.7969,  ...,  5.7500,  0.9062, -3.4844],
         [ 0.8398,  2.2812,  2.7969,  ...,  5.7500,  0.9062, -3.4844],
         [ 0.8398,  2.2812,  2.7969,  ...,  5.7500,  0.9062, -3.4844]]],
       dtype=torch.bfloat16, grad_fn=<ViewBackward0>)

In [66]:

apply_rotary_pos_emb(q, k, cos, sin)[0].transpose(1,2).reshape(1, 128, -1)


RuntimeError: The size of tensor a (2048) must match the size of tensor b (64) at non-singleton dimension 3